BiLSTM序列标注---盲汉翻译

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random # for dataset shuffling
import openpyxl # for recording experimental value
import copy # for data shuffling

In [ ]:
# GPU training
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
    print("number of device",torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("Running on the CPU")

https://blog.csdn.net/vivian_ll/article/details/93894151

In [2]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM以word_embeddings作为输入, 输出维度为 hidden_dim 的隐藏状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # 线性层将隐藏状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 各个维度的含义是 (num_layers*num_directions, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim).to(device),
                torch.zeros(1, 1, self.hidden_dim).to(device))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [3]:
#Making training data
original_data = []
t = open('./对应拼音标调.txt',"r", encoding='UTF8')
f = open("./训练集.txt", "rb")
#计数 数一共有多少行
lines = f.readlines()
tags = t.readlines()

# Training data: 6692(total number of sentences)*2(line&tag)
for index, line in enumerate(lines):
    line = lines[index].rstrip().split()
    tag = tags[index].rstrip().split()
    original_data.append((line, tag))

In [4]:
word_to_ix = {}
for sent, tags in original_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"0":0, "1":1, "2":2, "3":3, "4":4, ",":5, ".":6, "?":7,"!":8}

np.save('word_to_ix.npy',word_to_ix)
np.save('tag_to_ix.npy',tag_to_ix)

In [6]:
# Training the LSTM

#Monitor trianing results
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#Helper function to convert data into tensor
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).to(device)

# Accuracy recording in excel
workbook = openpyxl.Workbook()
worksheet = workbook.active



#Change parameters to reach better performances
for i in range(10):
    # record results.
    acc_repeat_train = ['acc_train']
    acc_repeat_val = ["acc_val"]
    loss_repeat_train = ["loss_repeat_train"]
    loss_repeat_val = ["loss_repeat_val"]

    # Repeat for three times in new model with different data. 
    for repeat in range(3):
        # Initialize the training model
        EMBEDDING_DIM = 60 
        HIDDEN_DIM = 10 + 20*i
        EPOCH=30
        model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
        model = model.to(device) # For GPU calculation
        loss_function = nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

        #Set data writer
        g = "./log/"+"Hidden_dim_"+str(EMBEDDING_DIM)+"_Embedding_dim_"+str(HIDDEN_DIM)+"repeat_"+str(repeat) 
        writer = SummaryWriter(g)

        #Shuffle data
        random.seed(repeat)
        training_data = copy.deepcopy(original_data) #Deep copy to make sure the original data is not influenced.
        random.shuffle(training_data)

        #Record accuracy


        for epoch in range(EPOCH):  
            print(epoch)
            training_acuracy = []
            validation_acuracy = []
            loss_list_train = []
            loss_list_validation = []
            for sentence, tags in tqdm(training_data[:5000]):
                # Clear Gradient because PyTorch accumulate it
                model.zero_grad()
                # Clear hidden state for LSTM, seperate them from the last instance.
                model.hidden = model.init_hidden()

                # 准备网络输入, 将其变为词索引的 Tensor 类型数据            
                sentence_in = prepare_sequence(sentence, word_to_ix)
                targets = prepare_sequence(tags, tag_to_ix)
                
                # Forward propagation
                tag_scores = model(sentence_in)

                # 第四步: 计算损失和梯度值, 通过调用 optimizer.step() 来更新梯度
                loss = loss_function(tag_scores, targets)
                a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
                training_acuracy.append(a)
                loss.backward()
                optimizer.step()
                loss_list_train.append(loss.item())
            
            #Validation
            for sentence, tags in tqdm(training_data[5000:6000]):
                model.zero_grad()
                model.hidden = model.init_hidden()
                # 准备网络输入, 将其变为词索引的 Tensor 类型数据
                sentence_in = prepare_sequence(sentence, word_to_ix)
                targets = prepare_sequence(tags, tag_to_ix)
                # 第三步: 前向传播.
                tag_scores = model(sentence_in)
                #计算损失
                a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
                validation_acuracy.append(a)
                loss = loss_function(tag_scores, targets)
                loss_list_validation.append(loss.item())
            
                
            #LSTM 可以做padding
            
            # Calculate error and accuracy
            # print(loss)
            loss_train= sum(loss_list_train)
            writer.add_scalar('Loss/training', loss_train, epoch)
            loss_val = sum(loss_list_validation)
            writer.add_scalar('Loss/validation', loss_val, epoch)
            #print(torch.argmax(tag_scores, dim=-1))
            # print("training_acuracy",sum(training_acuracy)/len(training_acuracy))
            acc_train = sum(training_acuracy)/len(training_acuracy)
            writer.add_scalar('Accuracy/training', acc_train, epoch)
            # print("validation_acuracy:",sum(validation_acuracy)/len(validation_acuracy))
            acc_val =sum(validation_acuracy)/len(validation_acuracy) 
            writer.add_scalar('Accuracy/validation', acc_val, epoch)

        acc_repeat_train.append(acc_train)
        acc_repeat_val.append(acc_val)
        loss_repeat_train.append(loss_train)
        loss_repeat_val.append(loss_val)         
            
        # 查看训练后的得分
        # with torch.no_grad():
        #     inputs = prepare_sequence(training_data[0][0], word_to_ix)
        #     tag_scores = model(inputs)

    worksheet.append(acc_repeat_train+acc_repeat_val+loss_repeat_train+loss_repeat_val)
writer.close()
workbook.save("Result.xlsx")

0


100%|██████████| 1000/1000 [00:03<00:00, 322.42it/s]


1


100%|██████████| 1000/1000 [00:04<00:00, 246.23it/s]


2


...

100%|██████████| 1000/1000 [00:04<00:00, 243.83it/s]


28


100%|██████████| 1000/1000 [00:03<00:00, 265.20it/s]


29


100%|██████████| 1000/1000 [00:03<00:00, 254.07it/s]
